1. Import Libraries

In [13]:
import pandas as pd
import csv

2. Load the EEG Data

In [14]:
import pandas as pd
import re
from collections import Counter  # (optional)

# -------- CONFIG --------
path = r"C:\Self Learning\Research Papers\UOW Research Papers\eSport Players with EEG Data\EEG Data\Jack\Jack_EPOC_147367_2023.02.15T16.20.09Z.md.mc.pm.fe.bp.csv"
expected_min_cols = 170
encodings_to_try = ["utf-8-sig", "utf-8", "latin-1"]

candidates = {
    ",":       {"is_regex": False},
    ";":       {"is_regex": False},
    "\t":      {"is_regex": False},
    "|":       {"is_regex": False},
    r"\s+":    {"is_regex": True},
}

# -------- STEP 1: decode a text sample for analysis --------
raw = None
text = None
used_encoding = None

for enc in encodings_to_try:
    try:
        with open(path, "rb") as f:
            raw = f.read()
        text = raw.decode(enc, errors="strict")
        used_encoding = enc
        break
    except Exception:
        continue

if text is None:
    raise RuntimeError("Could not decode file with utf-8-sig, utf-8, or latin-1.")

lines = text.splitlines()
sample_n = min(200, len(lines))
sample_lines = lines[:sample_n]

# -------- STEP 2: detect the header line and the best delimiter --------
def field_count(line, sep, is_regex):
    if not line.strip():
        return 0
    if is_regex:
        parts = re.split(sep, line.strip())
    else:
        parts = line.strip().split(sep)
    return len(parts)

best = None  # (max_fields, line_idx, sep, is_regex)

for idx, line in enumerate(sample_lines):
    if not line.strip():
        continue
    for sep, meta in candidates.items():
        cnt = field_count(line, sep, meta["is_regex"])
        if best is None or cnt > best[0]:
            best = (cnt, idx, sep, meta["is_regex"])

if best is None:
    raise RuntimeError("Could not identify any plausible header line.")

best_count, header_idx, best_sep, best_is_regex = best

if best_count < 50:
    print(f"⚠️ Detected only {best_count} fields on line {header_idx}. Continuing anyway.")

# -------- STEP 3: read the file from the detected header line --------
read_kwargs = {
    "encoding": used_encoding,
    "header": 0,                  # the first row after skiprows is the header
    "skiprows": header_idx,       # skip everything before the detected header
    "dtype": str,                 # keep 1:1 columns
    "quotechar": '"',
    "doublequote": True,
    "na_filter": False,
    # "on_bad_lines": "skip",     # uncomment if you hit malformed rows
}

if best_is_regex:
    read_kwargs["sep"] = best_sep
    read_kwargs["engine"] = "python"
else:
    read_kwargs["sep"] = best_sep
    read_kwargs["engine"] = "c"

df = pd.read_csv(path, **read_kwargs)

# -------- STEP 4: diagnostics and hard check --------
print(f"Detected encoding: {used_encoding}")
print(f"Detected header line index: {header_idx}")
print(f"Detected separator: {'REGEX ' if best_is_regex else ''}{best_sep}")
print(f"Shape: {df.shape}")

if df.shape[1] < expected_min_cols:
    hdr_preview = sample_lines[header_idx][:200].replace("\t", "\\t")
    print(f"⚠️ Columns < {expected_min_cols}. Header preview: {hdr_preview}")

assert df.shape[1] >= expected_min_cols, (
    f"Only {df.shape[1]} columns parsed; expected ≥ {expected_min_cols} (A..FN)."
)

print(df.columns.tolist()[:12])
print(df.head(3))


Detected encoding: utf-8-sig
Detected header line index: 1
Detected separator: ,
Shape: (104228, 170)
['Timestamp', 'OriginalTimestamp', 'EEG.Counter', 'EEG.Interpolated', 'EEG.AF3', 'EEG.F7', 'EEG.F3', 'EEG.FC5', 'EEG.T7', 'EEG.P7', 'EEG.O1', 'EEG.O2']
           Timestamp  OriginalTimestamp EEG.Counter EEG.Interpolated  \
0  1676478009.622028  1676478009.622347   21.000000         0.000000   
1  1676478009.629847  1676478009.630067   22.000000         0.000000   
2  1676478009.637667  1676478009.637887   23.000000         0.000000   

       EEG.AF3       EEG.F7       EEG.F3      EEG.FC5       EEG.T7  \
0  3831.794922  4352.307617  4011.794922  4031.281982  4431.282227   
1  3832.820557  4346.666504  4013.846191  4034.871826  4429.743652   
2  3829.743652  4347.692383  4010.256348  4038.461426  4428.717773   

        EEG.P7       EEG.O1       EEG.O2       EEG.P8       EEG.T8  \
0  4326.153809  4316.922852  4007.692383  4249.230957  3736.923096   
1  4328.717773  4317.948730  4007.17

In [15]:
data = df.copy()

3. Data Preprocessing

3.1. Extract the information of the dataset

In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104228 entries, 0 to 104227
Columns: 170 entries, Timestamp to POW.AF4.Gamma
dtypes: object(170)
memory usage: 135.2+ MB


In [17]:
data.columns

Index(['Timestamp', 'OriginalTimestamp', 'EEG.Counter', 'EEG.Interpolated',
       'EEG.AF3', 'EEG.F7', 'EEG.F3', 'EEG.FC5', 'EEG.T7', 'EEG.P7',
       ...
       'POW.F8.Theta', 'POW.F8.Alpha', 'POW.F8.BetaL', 'POW.F8.BetaH',
       'POW.F8.Gamma', 'POW.AF4.Theta', 'POW.AF4.Alpha', 'POW.AF4.BetaL',
       'POW.AF4.BetaH', 'POW.AF4.Gamma'],
      dtype='object', length=170)

In [18]:
pd.set_option('display.max_columns', None)  # Show every column (no truncation)
pd.set_option('display.width', 0)           # Let pandas auto-size the table to the notebook width
data.head()

,Timestamp,OriginalTimestamp,EEG.Counter,EEG.Interpolated,EEG.AF3,EEG.F7,EEG.F3,EEG.FC5,EEG.T7,EEG.P7,EEG.O1,EEG.O2,EEG.P8,EEG.T8,EEG.FC6,EEG.F4,EEG.F8,EEG.AF4,EEG.RawCq,EEG.Battery,EEG.BatteryPercent,MarkerIndex,MarkerType,MarkerValueInt,EEG.MarkerHardware,CQ.AF3,CQ.F7,CQ.F3,CQ.FC5,CQ.T7,CQ.P7,CQ.O1,CQ.O2,CQ.P8,CQ.T8,CQ.FC6,CQ.F4,CQ.F8,CQ.AF4,CQ.Overall,EQ.SampleRateQuality,EQ.OVERALL,EQ.AF3,EQ.F7,EQ.F3,EQ.FC5,EQ.T7,EQ.P7,EQ.O1,EQ.O2,EQ.P8,EQ.T8,EQ.FC6,EQ.F4,EQ.F8,EQ.AF4,MOT.CounterMems,MOT.InterpolatedMems,MOT.GyroX,MOT.GyroY,MC.Action,MC.ActionPower,MC.IsActive,PM.Engagement.IsActive,PM.Engagement.Scaled,PM.Engagement.Raw,PM.Engagement.Min,PM.Engagement.Max,PM.Excitement.IsActive,PM.Excitement.Scaled,PM.Excitement.Raw,PM.Excitement.Min,PM.Excitement.Max,PM.LongTermExcitement,PM.Stress.IsActive,PM.Stress.Scaled,PM.Stress.Raw,PM.Stress.Min,PM.Stress.Max,PM.Relaxation.IsActive,PM.Relaxation.Scaled,PM.Relaxation.Raw,PM.Relaxation.Min,PM.Relaxation.Max,PM.Interest.IsActive,PM.Interest.Scaled,PM.Interest.Raw,PM.Interest.Min,PM.Interest.Max,PM.Focus.IsActive,PM.Focus.Scaled,PM.Focus.Raw,PM.Focus.Min,PM.Focus.Max,FE.BlinkWink,FE.HorizontalEyesDirection,FE.UpperFaceAction,FE.UpperFaceActionPower,FE.LowerFaceAction,FE.LowerFaceActionPower,POW.AF3.Theta,POW.AF3.Alpha,POW.AF3.BetaL,POW.AF3.BetaH,POW.AF3.Gamma,POW.F7.Theta,POW.F7.Alpha,POW.F7.BetaL,POW.F7.BetaH,POW.F7.Gamma,POW.F3.Theta,POW.F3.Alpha,POW.F3.BetaL,POW.F3.BetaH,POW.F3.Gamma,POW.FC5.Theta,POW.FC5.Alpha,POW.FC5.BetaL,POW.FC5.BetaH,POW.FC5.Gamma,POW.T7.Theta,POW.T7.Alpha,POW.T7.BetaL,POW.T7.BetaH,POW.T7.Gamma,POW.P7.Theta,POW.P7.Alpha,POW.P7.BetaL,POW.P7.BetaH,POW.P7.Gamma,POW.O1.Theta,POW.O1.Alpha,POW.O1.BetaL,POW.O1.BetaH,POW.O1.Gamma,POW.O2.Theta,POW.O2.Alpha,POW.O2.BetaL,POW.O2.BetaH,POW.O2.Gamma,POW.P8.Theta,POW.P8.Alpha,POW.P8.BetaL,POW.P8.BetaH,POW.P8.Gamma,POW.T8.Theta,POW.T8.Alpha,POW.T8.BetaL,POW.T8.BetaH,POW.T8.Gamma,POW.FC6.Theta,POW.FC6.Alpha,POW.FC6.BetaL,POW.FC6.BetaH,POW.FC6.Gamma,POW.F4.Theta,POW.F4.Alpha,POW.F4.BetaL,POW.F4.BetaH,POW.F4.Gamma,POW.F8.Theta,POW.F8.Alpha,POW.F8.BetaL,POW.F8.BetaH,POW.F8.Gamma,POW.AF4.Theta,POW.AF4.Alpha,POW.AF4.BetaL,POW.AF4.BetaH,POW.AF4.Gamma
0,1676478009.622028,1676478009.622347,21.000000,0.000000,3831.794922,4352.307617,4011.794922,4031.281982,4431.282227,4326.153809,4316.922852,4007.692383,4249.230957,3736.923096,3976.410156,4506.153809,4238.974121,4191.282227,0.000000,1.000000,21.000000,,,,0.000000,4.000000,4.000000,4.000000,0.000000,0.000000,4.000000,4.000000,4.000000,2.000000,0.000000,4.000000,4.000000,4.000000,1.000000,0.000000,,,,,,,,,,,,,,,,,21.000000,0.000000,2180.000000,1479.000000,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1676478009.629847,1676478009.630067,22.000000,0.000000,3832.820557,4346.666504,4013.846191,4034.871826,4429.743652,4328.717773,4317.948730,4007.179443,4254.871582,3736.923096,3973.333252,4505.641113,4236.922852,4197.948730,0.000000,1.000000,21.000000,,,,0.000000,4.000000,4.000000,4.000000,0.000000,0.000000,4.000000,4.000000,4.000000,2.000000,0.000000,4.000000,4.000000,4.000000,1.000000,0.000000,,,,,,,,,,,,,,,,,22.000000,0.000000,2178.000000,1480.000000,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,0.000000,0.000000,64.000000,0.204746,1.000000,0.000000,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,1676478009.637667,1676478009.637887,23.000000,0.000000,3829.743652,4347.692383,4010.256348,4038.461426,4428.717773,4334.358887,4318.461426,4007.179443,4256.922852,3736.923096,3975.897461,4508.717773,4246.666504,4203.589844,0.000000,1.000000,21.000000,,,,0.000000,4.000000,4.000000,4.000000,0.000000,0.000000,4.000000,4.000000,4.000000,2.000000,0.000000,4.000000,4.000000,4.000000,1.000000,0.000000,,,,,,,,,,,,,,,,,23.000000,0.000000,2179.000000,1479.000000,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3,1676478009.645486,1676478009.645607,24.000000,0.000000,3817.

3.2. Keep only the columns from E-R

In [19]:
# Keep only columns E..R (Excel letters). 
# A=0,B=1,C=2,D=3,E=4,...,R=17 (0-based). iloc stop is exclusive, so use 18.
start_idx = 4           # E
stop_idx_exclusive = 18 # R + 1

# Clamp the stop index in case the file has fewer columns
stop_idx_exclusive = min(stop_idx_exclusive, data.shape[1])

data = data.iloc[:, start_idx:stop_idx_exclusive]

# (Optional) quick check
print(data.shape)

(104228, 14)


In [20]:
data.head()

,EEG.AF3,EEG.F7,EEG.F3,EEG.FC5,EEG.T7,EEG.P7,EEG.O1,EEG.O2,EEG.P8,EEG.T8,EEG.FC6,EEG.F4,EEG.F8,EEG.AF4
0,3831.794922,4352.307617,4011.794922,4031.281982,4431.282227,4326.153809,4316.922852,4007.692383,4249.230957,3736.923096,3976.410156,4506.153809,4238.974121,4191.282227
1,3832.820557,4346.666504,4013.846191,4034.871826,4429.743652,4328.717773,4317.948730,4007.179443,4254.871582,3736.923096,3973.333252,4505.641113,4236.922852,4197.948730
2,3829.743652,4347.692383,4010.256348,4038.461426,4428.717773,4334.358887,4318.461426,4007.179443,4256.922852,3736.923096,3975.897461,4508.717773,4246.666504,4203.589844
3,3817.948730,4340.512695,4002.564209,4033.333252,4428.205078,4333.846191,4316.410156,4007.692383,4251.794922,3736.923096,3975.384521,4503.077148,4248.205078,4198.974121
4,3809.743652,4327.179688,3993.333252,4028.718018,4426.666504,4325.641113,4314.871582,4005.128174,4249.230957,3736.923096,3968.718018,4494.358887,4235.384766,4192.820313


* First row is the header row. so it will not be applied in the simulation

3.3. Extract EEG data from sample (row) number 23,040 to the row number 76,800.

In [21]:
# -------- STEP 5: extract rows 23,040..76,800 (inclusive, 1-based) --------
TARGET_N = 76_800
start_row_1based = 23_040
end_row_1based   = 76_800

# Convert to 0-based, end exclusive for iloc
start_iloc = max(0, start_row_1based - 1)
n_rows = data.shape[0]
end_iloc_exclusive = min(n_rows, end_row_1based)  # 1-based inclusive -> 0-based exclusive

# Final dataset
data_sliced = data.iloc[start_iloc:end_iloc_exclusive].copy()

# Reporting
print(f"Total rows in data: {n_rows}")
if n_rows == TARGET_N:
    print("✅ Dataset has exactly 76,800 rows.")
elif n_rows > TARGET_N:
    print(f"ℹ️ Dataset has MORE than 76,800 rows: {n_rows} (+{n_rows - TARGET_N}). "
          "Printing rows beyond 76,800 as well.")
else:
    print(f"⚠️ Dataset has FEWER than 76,800 rows: {n_rows}. Slice may be shorter than requested.")

print(f"Requested rows (1-based): {start_row_1based}..{end_row_1based}")
print(f"Sliced (0-based iloc): {start_iloc}:{end_iloc_exclusive}")
print(f"data_sliced shape: {data_sliced.shape}")
if data_sliced.empty:
    print("⚠️ Slice is empty (start index is beyond available rows).")
else:
    print("Preview of requested slice:")
    print(data_sliced.head())

# If there are rows beyond 76,800, report and preview them too
if n_rows > TARGET_N:
    extra_start = TARGET_N  # 0-based iloc start for "beyond" section
    extra_end = n_rows
    extra = data.iloc[extra_start:extra_end].copy()
    print(f"\nExtra rows beyond 76,800: {n_rows - TARGET_N}")
    print(f"Extra slice (0-based iloc): {extra_start}:{extra_end} | shape: {extra.shape}")
    

Total rows in data: 104228
ℹ️ Dataset has MORE than 76,800 rows: 104228 (+27428). Printing rows beyond 76,800 as well.
Requested rows (1-based): 23040..76800
Sliced (0-based iloc): 23039:76800
data_sliced shape: (53761, 14)
Preview of requested slice:
           EEG.AF3       EEG.F7       EEG.F3      EEG.FC5       EEG.T7  \
23039  3694.358887  4258.461426  3981.538574  4424.102539  4428.205078   
23040  3689.743652  4248.205078  3982.051270  4422.051270  4428.205078   
23041  3692.307617  4248.205078  3987.179443  4422.563965  4427.692383   
23042  3692.820557  4251.794922  3987.179443  4422.563965  4426.666504   
23043  3692.307617  4248.717773  3985.641113  4423.077148  4428.205078   

            EEG.P7       EEG.O1       EEG.O2       EEG.P8       EEG.T8  \
23039  4362.051270  4348.205078  3990.256348  4260.512695  3737.948730   
23040  4354.871582  4349.230957  3998.974365  4261.538574  3737.948730   
23041  4364.615234  4357.948730  4010.769287  4262.051270  3737.948730   
23042  

3.4. Keep last 53000 rows

In [22]:
# -------- STEP 6: keep only the last 53,000 rows from data_sliced --------
TARGET_53K = 53_000

n_curr = data_sliced.shape[0]
if n_curr >= TARGET_53K:
    # number of rows to remove from the start
    n_drop = n_curr - TARGET_53K
    data_last_53k = data_sliced.iloc[-TARGET_53K:].copy()
    print(f"Trimmed {n_drop} rows from the start to keep the last {TARGET_53K}.")
else:
    # not enough rows to trim; keep as-is
    n_drop = 0
    data_last_53k = data_sliced.copy()
    print(f"Dataset has only {n_curr} rows (< {TARGET_53K}); keeping all.")

print(f"data_last_53k shape: {data_last_53k.shape}")

# (Optional) reset the index if you want a clean 0..N-1 index
data_last_53k = data_last_53k.reset_index(drop=True)


Trimmed 761 rows from the start to keep the last 53000.
data_last_53k shape: (53000, 14)


In [23]:
data_last_53k.head()

,EEG.AF3,EEG.F7,EEG.F3,EEG.FC5,EEG.T7,EEG.P7,EEG.O1,EEG.O2,EEG.P8,EEG.T8,EEG.FC6,EEG.F4,EEG.F8,EEG.AF4
0,3711.794922,4275.384766,3983.589844,4422.563965,4430.256348,4361.538574,4361.025879,4007.179443,4290.256348,3738.974365,3911.794922,4479.487305,4184.615234,4270.769043
1,3709.230713,4272.820313,3979.487061,4420.000000,4430.769043,4355.384766,4358.461426,3994.871826,4280.000000,3738.461426,3910.256348,4474.871582,4183.077148,4268.205078
2,3722.564209,4282.563965,3989.743652,4423.077148,4428.717773,4366.666504,4367.692383,3996.923096,4286.153809,3738.974365,3916.923096,4473.333496,4194.358887,4271.282227
3,3726.666748,4288.717773,3994.871826,4425.641113,4429.743652,4372.820313,4365.128418,4000.512939,4289.230957,3738.461426,3923.076904,4477.948730,4193.333496,4275.384766
4,3721.538574,4287.179688,3995.897461,4423.077148,4430.256348,4366.666504,4357.948730,3998.461426,4282.563965,3738.974365,3926.153809,4483.077148,4190.769043,4275.384766


3.5. Create csv file from filtered data

In [25]:
import os
import csv  # you already imported this above

# Ensure you've created `data_last_53k` from the previous step
# and that `path` points to your original CSV.

output_path = os.path.join(os.path.dirname(path), "jack_eeg_modified.csv")

data_last_53k.to_csv(
    output_path,
    index=False,
    encoding="utf-8-sig",   # keeps Excel happy with UTF-8 BOM
    quoting=csv.QUOTE_MINIMAL
    # line_terminator="\n", # optional: normalize newlines
)

print(f"✅ Saved: {output_path} | shape: {data_last_53k.shape}")


✅ Saved: C:\Self Learning\Research Papers\UOW Research Papers\eSport Players with EEG Data\EEG Data\Jack\jack_eeg_modified.csv | shape: (53000, 14)


4. Create 2D array

4.1. Convert to an array

In [26]:
import numpy as np

# If you want raw strings exactly as in the CSV:
arr = data_last_53k.to_numpy(copy=True)
assert arr.shape == (53000, 14)

In [27]:
arr.shape

(53000, 14)

4.2. Transpose the array

In [28]:
# Transpose (returns a view when possible)
arr_T = arr.T
print(arr_T.shape)   # (14, 53000)

(14, 53000)


5. Segement to (14,200) shape

In [29]:
import numpy as np

seg_len = 200
channels, total_samples = arr_T.shape
n_segments = total_samples // seg_len  # 53000 // 200 = 265

# (Optional) trim if not perfectly divisible — here it is, but this keeps it robust
usable = n_segments * seg_len
arr_T_trim = arr_T[:, :usable]

# Reshape to (channels, segments, seg_len) then put segments first → (segments, channels, seg_len)
segments = arr_T_trim.reshape(channels, n_segments, seg_len).transpose(1, 0, 2)

print("segments.shape:", segments.shape)  # (265, 14, 200)


segments.shape: (265, 14, 200)


6. Add target variable

In [30]:
import numpy as np

y = np.ones(265, dtype=np.int64)   # shape (265,)

In [31]:
y.shape

(265,)

In [32]:
y[:10]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

7. Save the arrays

7.1. Save the segments

In [33]:
import os
import numpy as np
import csv  # if you're also writing CSVs

# Assume: `path` points to your original CSV and `segments` is (265, 14, 200)
# (Optional) y is your labels vector (265,)

out_dir = os.path.dirname(path)

# ---- Save segments as a NumPy array (.npy) ----
adam_eeg_path = os.path.join(out_dir, "jack_eeg.npy")
np.save(adam_eeg_path, segments)
print(f"✅ Saved segments → {adam_eeg_path} | shape: {segments.shape}")

# ---- (Optional) Save labels next to it ----



✅ Saved segments → C:\Self Learning\Research Papers\UOW Research Papers\eSport Players with EEG Data\EEG Data\Jack\jack_eeg.npy | shape: (265, 14, 200)


7.2. Save the target array

In [34]:
y = np.ones(segments.shape[0], dtype=np.int64)
y_path = os.path.join(out_dir, "jack_eeg_labels.npy")
np.save(y_path, y)
print(f"✅ Saved labels   → {y_path} | shape: {y.shape}")

✅ Saved labels   → C:\Self Learning\Research Papers\UOW Research Papers\eSport Players with EEG Data\EEG Data\Jack\jack_eeg_labels.npy | shape: (265,)
